In [ ]:
import pymc as pm
import numpy as np
import pandas as pd
from string import ascii_lowercase
from pathlib import Path
from urllib.parse import urlparse 
from concurrent.futures import ThreadPoolExecutor 
import multiprocessing 
from queue import Queue, Empty 
import pickle
import requests
from elasticsearch import Elasticsearch
from pathlib import Path
import os
import json
import pickle

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [1]:
def run_indexer(self):
    self.es_client.options(ignore_status=[400, 404]).indices.delete(index='custom')
    index_body = {
        "settings": {
            "similarity": {
                "custom_similarity": {
                    "type": "scripted",
                    "script": {
                        "source": """
                            double idf = Math.log((field.docCount + 1.0) / (term.docFreq + 1.0)) + 1.0;
                            return idf;
                        """
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "text": {
                    "type": "text",
                    "similarity": "custom_similarity"
                }
            }
        }
    }
    self.es_client.options(ignore_status=400).indices.create(index='custom', body=index_body)


In [2]:
from flask import Flask, request
import time

In [3]:
from flask import Flask, request
from elasticsearch import Elasticsearch
import pandas as pd
import time

# Initialize Flask app
app = Flask(__name__)

# Connect to Elasticsearch
app.es_client = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "D*d4-0+Kl+lxfbbzh5ut"),  # Replace with your password
    ca_certs="~/http_ca.crt"  # Adjust the path to your certificate if needed
)

# Define the search endpoint
@app.route('/search_custom', methods=['GET'])
def search_es():
    # Record start time for query execution
    start = time.time()
    response_object = {'status': 'success'}
    
    # Get the search query from request arguments
    argList = request.args.to_dict(flat=False)
    query_term = argList['query'][0]
    
    # Perform the search on Elasticsearch
    results = app.es_client.search(
        index='simple',
        source_excludes=['url_lists'],  # Exclude unnecessary fields
        size=100,  # Limit the results to 100
        query={"match": {"text": query_term}}  # Match query on the 'text' field
    )
    
    # Record end time for query execution
    end = time.time()
    total_hit = results['hits']['total']['value']
    
    # Prepare results in a DataFrame
    results_df = pd.DataFrame(
        [
            [
                hit["_source"]['title'],  # Title of the document
                hit["_source"]['url'],  # URL of the document
                hit["_source"]['text'][:100],  # First 100 characters of text
                hit["_score"]  # Relevance score
            ]
            for hit in results['hits']['hits']
        ],
        columns=['title', 'url', 'text', 'score']
    )
    
    # Populate the response object
    response_object['total_hit'] = total_hit
    response_object['results'] = results_df.to_dict('records')
    response_object['elapse'] = end - start  # Query execution time
    
    return response_object
app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Jan/2025 16:39:11] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [27/Jan/2025 16:39:30] "GET /search_custom?query=school HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 16:44:03] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [27/Jan/2025 16:44:12] "GET /search_manual?query=school HTTP/1.1" 404 -
127.0.0.1 - - [27/Jan/2025 16:44:18] "GET /search_manual?query=school HTTP/1.1" 404 -
127.0.0.1 - - [27/Jan/2025 16:46:07] "GET /search_manual?query=school HTTP/1.1" 404 -
127.0.0.1 - - [27/Jan/2025 16:46:07] "GET /search_manual?query=school HTTP/1.1" 404 -
127.0.0.1 - - [27/Jan/2025 16:46:08] "GET /search_manual?query=school HTTP/1.1" 404 -
127.0.0.1 - - [27/Jan/2025 16:46:08] "GET /search_manual?query=school HTTP/1.1" 404 -
127.0.0.1 - - [27/Jan/2025 16:47:01] "GET /search_manual?query=school HTTP/1.1" 404 -
127.0.0.1 - - [27/Jan/2025 16:47:01] "GET /search_manual?query=school HTTP/1.1" 404 -
127.0.0.1 - - [27/Jan/2025 16:47:01] "GET /search

The result from simple = 0.135 score and the custom score = 0.12